# 1. Install depencencies

In [ ]:
## Uncomment for using in Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install 'transformers[torch]' faker transformers torch accelerate fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.6 MB/s eta 0:00:00


In [ ]:
# from faker import Faker
import pathlib
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, DistilBertForSequenceClassification, DistilBertTokenizerFast
import torch

# 2. Create synthetic text data with  personal data in the half of data points

## Uncomment next two folders to generate and store synthetic data


In [ ]:
# from create_synthetic_dataset import generate_synthetic_data

In [ ]:
# %%time
# # # Directory to store datasets
# # datasets = 'ADJUST PATH TO YOUR DATASET'
# pathlib.Path(datasets).mkdir(exist_ok=True, parents=True)

# # Example of usage
# num_samples=100
# synthetic_data = generate_synthetic_data(num_samples)
# # Saving data to a CSV file
# file_name = f'{datasets}/synthetic_data_{int(num_samples/1_000_000)}m.parquet'
# synthetic_data.to_parquet(file_name, compression='gzip', index=False)

# 3. Work with synthetic_data

---



In [ ]:
# Load the synthetic data
file_name = '/content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/datasets/synthetic_data_10m.parquet'
# file_name = 'ADJUST PATH TO YOU DATASET'
df = pd.read_parquet(file_name)

In [ ]:
# Check the data
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9694936 entries, 0 to 9694935
Data columns (total 2 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   text               9694936 non-null  object
 1   has_personal_data  9694936 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 147.9+ MB


In [ ]:
df.head(10)

,text,has_personal_data
0,Tend recently daughter high generation soldier...,0
1,Money that understand become. Quite even syste...,0
2,Factor reality tough cup be win. Happen invest...,0
3,Heavy could top safe friend. Great hour popula...,0
4,"587 Brian Ramp Apt. 558\nBasshaven, IA 05960",1
5,Reginald Watson jason36@mitchell-pugh.com 451-...,1
6,"4611 Tony Plaza\nNew Carolynport, VA 40254 202...",1
7,Hotel over leave realize remember somebody eas...,0
8,Agree agent red claim. Party hit option there ...,0
9,311-90-3647 yadams@gmail.com 9724 Harris Mount...,1


In [ ]:
# Check whether the dataset is balanced or not
df[df['has_personal_data']== 0].shape[0] == df[df['has_personal_data']== 1].shape[0]

True

## 3.1 Select subset with desired size of num_samples. We used totally 1 mln datapoints: 500k with fake personal data and 500k without it

In [ ]:
# Select rows where has_personal_data is 1
has_personal_data_1 = df[df['has_personal_data'] == 1]

# Select rows where has_personal_data is 0
has_personal_data_0 = df[df['has_personal_data'] == 0]

# Determine the minimum number of rows for each group.
num_samples = 500_000

# Sample an equal number of rows from each group
balanced_df = pd.concat([has_personal_data_0.sample(n=num_samples, random_state=421),
                             has_personal_data_1.sample(n=num_samples , random_state=421)])

# Reset index if needed and  Shuffle the DataFrame
balanced_df = balanced_df.sample(frac=1, random_state=421).reset_index(drop=True)
balanced_df.head(10)

,text,has_personal_data
0,Enjoy of woman computer. Trip soon law claim s...,0
1,Available color simple Democrat good then stro...,0
2,Andrew Mathis lindawatkins@gmail.com 2003-05-0...,1
3,sbridges@hotmail.com Kathleen Robertson Nothin...,1
4,Together ever in spend little argue some.\nLis...,0
5,Two woman face their language hour. Like bill ...,0
6,John Baker 752-234-8279,1
7,Like relationship where represent. Give nation...,0
8,1991-02-10 joyceralph@gmail.com Race between c...,1
9,Create check begin model feeling throughout. F...,0


In [ ]:
# Check whether the dataset is balanced or not
balanced_df[balanced_df['has_personal_data']== 0].shape[0] == balanced_df[balanced_df['has_personal_data']== 1].shape[0]

True

In [ ]:
# # If dataset is imbalanced,
## a) Sepparate the DataFrame into two based on the 'has_personal_data' column
# df_0 = df[df['has_personal_data'] == 0]
# df_1 = df[df['has_personal_data'] == 1]

# # b) Sample from the larger class to match the size of the smaller class
# df_0_balanced = df_0.sample(n=len(df_1), replace=False, random_state=42)

# # c) Concatenate the balanced subsets
# balanced_df = pd.concat([df_0_balanced, df_1])

# # d) Shuffle the concatenated DataFrame and export it
# balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
# file_name = 'synthetic_data_10m_balanced.parquet'
# balanced_df.to_parquet(file_name, index=False, compression='gzip')


## 3.2 Split data into train, validation, and test sets

In [ ]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(balanced_df, test_size=0.2, random_state=42)
test_df, validation_df = train_test_split(test_df, test_size=0.5, random_state=42)
train_df

,text,has_personal_data
566853,848-98-7571 danielyork@gmail.com,1
382311,001-900-404-6417 9252 Seth Corners Suite 538\n...,1
241519,Participant receive course every agreement tod...,0
719220,Television plan must clear. Window change mate...,0
905718,Address perform end. Man already themselves go...,0
...,...,...
259178,dawnbruce@edwards.info 99639 Sarah Summit\nEas...,1
365838,For employee ball American training of will. D...,0
131932,Modern current although area suddenly. Worry d...,0
671155,Middle option house city tough. Born sure fini...,0


In [ ]:
test_df

,text,has_personal_data
803899,Kitchen adult home stop. Republican would inde...,0
624862,Pull Democrat piece join decide left others. W...,0
666323,Production almost responsibility. Not responsi...,0
742056,Religious family move as sister. Course still ...,0
896804,Stand produce once difference writer yeah. Cam...,0
...,...,...
146165,Seven land ball pattern social back. Picture M...,0
927253,913-562-1286x708,1
392080,Tyler Shelton vbrown@moreno.com,1
669591,Steven Chapman Mrs. Alicia Lopez MD 83077 Brad...,1


In [ ]:
validation_df

,text,has_personal_data
333823,Notice price by race final idea.\nWhile choose...,0
654420,Speak happy arrive though. Power street person...,0
744810,sullivankeith@gmail.com,1
289997,william69@carr.org 1977-04-18 137 Johnson Driv...,1
518932,Billy Garcia 42291 Anthony Port Apt. 121\nAnde...,1
...,...,...
680785,edward15@gmail.com,1
841578,"506 Bush Throughway Suite 166\nFieldsburgh, NJ...",1
922188,Stephen Holloway +1-971-246-4483x90144 1599 Ba...,1
674157,Cell many reduce determine. Our best if pictur...,0


# 4. Train model on the synthetic data

In [ ]:
%%time
# Folder to store results of training and testing
num_samples=1_000_000
EXP_DIR = f'/content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_{int(num_samples/1000000)}m'
# EXP_DIR = 'ADJUST PATH TO YOU FOLDER'
pathlib.Path(EXP_DIR).mkdir(exist_ok=True, parents=True)
EXP_DIR

CPU times: user 1.18 ms, sys: 43 µs, total: 1.22 ms
Wall time: 1.38 ms


'/content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m'

In [ ]:
%%time
# Initialize the BERT tokenizer and model
model_name = 'distilbert-base-uncased'
# Load the BERT tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

CPU times: user 207 ms, sys: 13.4 ms, total: 221 ms
Wall time: 4.22 s


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # binary classification

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Use Dataloader to upload data

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx]['text']
        label = self.df.iloc[idx]['has_personal_data']

        encoding = self.tokenizer(text, padding='max_length', truncation=True, return_tensors='pt', max_length=256)

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label)
        }

# Create instances of the custom dataset
train_dataset = CustomDataset(train_df, tokenizer)
validation_dataset = CustomDataset(validation_df, tokenizer)
test_dataset = CustomDataset(test_df, tokenizer)

In [ ]:
# Get the CUDA device capability
cuda_capability = torch.cuda.get_device_capability(0)
print(cuda_capability )
# Check if CUDA is available and the major version is greater than or equal to 7
fp16 = torch.cuda.is_available() and cuda_capability[0] >= 7
fp16

True

In [ ]:
# Set up the training arguments with --load_best_model_at_end
training_args = TrainingArguments(
    output_dir=f'{EXP_DIR}/results',
    num_train_epochs=20,
    per_device_train_batch_size=128, # Adjust according conditions
    fp16=fp16,
    evaluation_strategy="steps",  # Change evaluation strategy to "steps"
    logging_dir=f'{EXP_DIR}/logs',
    logging_steps=100,
    do_train=True,
    do_eval=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=3,
)


# Trainer initialization with the EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.02)],
)

In [ ]:
%%time
# Train the model
trainer.train()

# Evaluate the model on the test dataset
test_results = trainer.evaluate(test_dataset)  # Pass test_dataset

# Print test results
print(test_results)

Step,Training Loss,Validation Loss
100,0.039900,0.000432
200,0.000200,0.000158
300,0.001800,0.000101
400,0.000100,0.000073
500,0.000000,0.000062
600,0.000300,0.000018
700,0.000000,0.000282
800,0.000900,0.000113


{'eval_loss': 2.3701109967078082e-05, 'eval_runtime': 289.2591, 'eval_samples_per_second': 345.711, 'eval_steps_per_second': 43.214, 'epoch': 0.13}
CPU times: user 53min 53s, sys: 13.5 s, total: 54min 6s
Wall time: 56min 34s


In [ ]:
# Save trained model
output_model_dir = f'{EXP_DIR}/model_{int(num_samples/1000000)}m'
# output_model_dir = 'ADJUST PATH TO YOU FOLDER'
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

('/content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m/model_1m/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m/model_1m/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m/model_1m/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m/model_1m/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m/model_1m/tokenizer.json')

In [ ]:
!zip -r "/content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m.zip" "/content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m"

  adding: content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m/ (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m/results/ (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m/results/checkpoint-500/ (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m/results/checkpoint-500/config.json (deflated 46%)
  adding: content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m/results/checkpoint-500/model.safetensors (deflated 8%)
  adding: content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m/results/checkpoint-500/training_args.bin (deflated 51%)
  adding: content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m/results/checkpoint-500/optimizer.pt (deflated 34%)
  adding: content/drive/MyDrive/Colab Notebooks/projects/personal_data_detector/run_1m/results/checkpoint-500

# 5. Make a classificator to distinguish weather text contains persona data or not

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the trained model and tokenizer
output_model_dir = 'run_1m/model_1m/'
# model_name = 'ADJUST PATH TO YOUR MODEL HERE'
model_name = output_model_dir
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

# Function to classify input text
def classify_personal_data(input_text):
    # Tokenize the input text
    # inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt")
    inputs = tokenizer(input_text, padding='max_length', truncation=True, return_tensors='pt', max_length=256)
    # Make a prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted label (0 or 1)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    print(predicted_label)
    return "The input text contains personal data." if predicted_label else "The input text does not contain personal data."

In [ ]:
# Example usage
input_text = "John Doe lives at 123 Main Street New York."
result = classify_personal_data(input_text)
print(result)

1
The input text contains personal data.


In [ ]:
classify_personal_data('There is no personal data at all!!!')

0


'The input text does not contain personal data.'

In [ ]:
classify_personal_data(input_text='I go to the nearest shop to buy milk')

0


'The input text does not contain personal data.'

In [ ]:
classify_personal_data(input_text="abra kadabra it's not personal data")

1


'The input text contains personal data.'

In [ ]:
classify_personal_data(input_text="the moon in the sky at night is very beautiful")

0


'The input text does not contain personal data.'

In [ ]:
classify_personal_data(input_text="We live on the planet earth. ")

0


'The input text does not contain personal data.'

In [ ]:
classify_personal_data(input_text="Sherlock Holms, London, +12345678")

1


'The input text contains personal data.'

In [ ]:
classify_personal_data(input_text="Woody Woodpecker is and animated hero. It's name is fictiona and it's not personal data")

1


'The input text contains personal data.'

In [ ]:
classify_personal_data(input_text="Is it true that you can classify wether sentence contains personal data or not + 55 and for example email?")

0


'The input text does not contain personal data.'